In [1]:
import tensorflow as tf
from tensorflow import keras
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

2024-11-01 23:09:30.848465: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-01 23:09:30.876082: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-01 23:09:31.133466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 23:09:31.133556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 23:09:31.154621: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
model = keras.models.load_model('model_epoch_17_val_acc_0.9615.h5')

2024-11-01 23:09:33.939672: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-01 23:09:33.940174: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
from keras.preprocessing.image import load_img
from keras.applications.resnet import preprocess_input
import numpy as np

In [4]:
img = load_img('./sp1-P1.jpg', target_size=(224, 224))

In [5]:
x = np.array(img)
X = np.array([x])
X.shape

(1, 224, 224, 3)

In [6]:
X = preprocess_input(X)
pred = model.predict(X)
pred

1/1 [==============================] - 1s 780ms/step


array([[0.06517317, 0.93482685]], dtype=float32)

In [7]:
pred = {"prediction": pred[0][1]}
pred

{'prediction': 0.93482685}

# Convert Keras to TF-Lite

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('model_hw_newhandpd_aug-ilum_rasnet50.tflite', 'wb') as f_out:
    f_out.write(tflite_model)


2024-11-01 23:09:53.175404: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-11-01 23:09:53.175463: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-11-01 23:09:53.179782: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpvz_252d5
2024-11-01 23:09:53.200909: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-11-01 23:09:53.200950: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpvz_252d5
2024-11-01 23:09:53.245684: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-11-01 23:09:53.270475: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-11-01 23:09:53.815470: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpvz_252d5
2024-11

In [9]:
import tensorflow.lite as tflite

In [10]:
interpreter = tflite.Interpreter(model_path='model_hw_newhandpd_aug-ilum_rasnet50.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [11]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [12]:
pred = {"prediction": preds[0][1]}
pred

{'prediction': 0.9348267}

# Removing TF dependency

In [13]:
from PIL import Image

In [14]:
with Image.open('./sp1-P1.jpg') as img:
    img = img.resize((224, 224), Image.NEAREST)

In [15]:
def preprocess_input(x):
    x = x[..., ::-1]
    mean = [103.939, 116.779, 123.68]
    x[..., 0] -= mean[0]
    x[..., 1] -= mean[1]
    x[..., 2] -= mean[2]
    return x

In [16]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [17]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
pred = {"prediction": preds[0][1]}
pred

{'prediction': 0.9348267}

In [18]:
!pip install keras-image-helper

In [19]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/


In [20]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [21]:
interpreter = tflite.Interpreter(model_path='model_hw_newhandpd_aug-ilum_rasnet50.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [22]:
preprocessor = create_preprocessor('resnet50', target_size=(224, 224))

In [23]:
url = 'https://github.com/andyathsid/parked-ml-backend/blob/dev/hand-writing-prediction-service/sp1-P1.jpg?raw=true'
X = preprocessor.from_url(url)

HTTPError: HTTP Error 404: Not Found

In [5]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
pred = {"prediction": preds[0][1]}
pred

{'prediction': 0.90632087}